In [11]:
# -*-coding:utf-8 -*-
# Edited by bighead 19-2-19

import numpy as np
import pandas as pd
import cv2
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
def binaryzation(img):
    cv_img = img.astype(np.uint8)
    cv2.threshold(cv_img,50,1,cv2.THRESH_BINARY_INV,cv_img)
    return cv_img

In [13]:
def train(trainset, train_labels):
    prior_probability = np.zeros(class_num)
    conditional_probability = np.zeros((class_num, feature_len, 2))
    
    for i in range(len(train_labels)):
        img = binaryzation(trainset[i])
        label = train_labels[i]
        
        prior_probability[label] += 1
        
        for j in range(feature_len):
            conditional_probability[label][j][img[j]] += 1
    
    for i in range(class_num):
        for j in range(feature_len):
            pix_0 = conditional_probability[i][j][0]
            pix_1 = conditional_probability[i][j][1]
            
            probalitity_0 = (float(pix_0)/float(pix_0+pix_1)) * 1000000 + 1
            probalitity_1 = (float(pix_1)/float(pix_0+pix_1)) * 1000000 + 1
            
            conditional_probability[i][j][0] = probalitity_0
            conditional_probability[i][j][1] = probalitity_1
    
    return prior_probability, conditional_probability

In [14]:
def calculate_probability(img, label):
    probability = int(prior_probability[label])
    
    for i in range(len(img)):
        probability *= int(conditional_probability[label][i][img[i]])
    
    return probability

In [15]:
def Predict(testset, prior_probility, conditional_probability):
    predict = []
    
    for img in testset:
        
        img = binaryzation(img)
        
        max_label = 0
        max_probability = calculate_probability(img, 0)
        
        for j in range(1, 10):
            probability = calculate_probability(img, j)
            
            if max_probability < probability:
                max_label = j
                max_probability = probability
        
        predict.append(max_label)
    
    return np.array(predict)

In [16]:
class_num = 10
feature_len = 784

In [18]:
print 'Start read data'
time_1 = time.time()

raw_data = pd.read_csv('../data/train.csv', header=0)
data = raw_data.values

imgs = data[:, 1:]
labels = data[:, 0]

features_train, features_test, labels_train, labels_test = train_test_split(imgs, labels, test_size=0.33, random_state=2333)
time_2 = time.time()

print 'read data cost: {} seconds'.format(time_2 - time_1)

print 'start training'
prior_probability, conditional_probability = train(features_train, labels_train)
time_3 = time.time()
print 'training cost: {} seconds'.format(time_3-time_2)

print 'start predicting'
test_predict = Predict(features_test, prior_probility=prior_probability, conditional_probability=conditional_probability)
time_4 = time.time()
print 'testing cost: {} seconds'.format(time_4-time_3)

score = accuracy_score(labels_test, test_predict)
print 'accruacy score is {}'.format(score)

Start read data
read data cost: 4.78487610817 seconds
start training
training cost: 45.6151919365 seconds
start predicting
testing cost: 293.034191847 seconds
accruacy score is 0.8386002886
